In [1]:
from bs4 import BeautifulSoup
import re
import numpy as np
import os
from tqdm.notebook import tqdm
import malaya

2021-10-09 11:26:38.926044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-09 11:26:38.926143: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/l12-o-0-554/Documents/virtualenvs/mPunct/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


In [17]:
#corrector = malaya.spell.probability()
normalizer = malaya.normalize.normalizer(date=False,time=False)

In [22]:
def clean_text(text):
    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    
    #reg = "(?<=[a-zA-Z])-(?=[a-zA-Z]{2,})"  ## comment this out please! no replacing '-'s for malay
    #r = re.compile(reg, re.DOTALL)
    #text = r.sub(' ', text)
    text = re.sub(r'\[.+?\]','',text)
    text = re.sub(r'( -| - |- )','',text)
    text = re.sub(r'(?<=[a-zA-Z])\.(?=[a-zA-Z])','. ',text)
    text = re.sub(r'(?<=[a-zA-Z])\?(?=[a-zA-Z])','? ',text)
    text = re.sub(r'(?<=[a-zA-Z])\,(?=[a-zA-Z])',', ',text)
#     text = text.replace('-', ',')
    text = text.replace(';', '.')    # replace symbols with the most relevant counterparts
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')
    text = text.replace('♪ ♪',',')
    text = text.replace('♪','')
    text = text.replace('...', '')
    text = text.replace(r'.\s*\"', ',')
    text = text.replace(r',\s*"',', ')
    text = text.replace(r'"', ',')

    text = re.sub(r'--\s?--', '', text) # replace --   -- to ''
    text = re.sub(r'\s+', ' ', text)    # replace all extra whitespaces
    text = re.sub(r'[⁰¹²³⁴⁵⁶⁷⁸⁹]',"",text)
    
    text = re.sub(r',\s?,', ',', text)  # merge commas separating only whitespace
    text = re.sub(r',\s?\.', '.', text) # , . -> ,
    text = re.sub(r'(?<=[a-zA-Z0-9]),(?=[a-zA-Z0-9])',', ',text) # say,you -> say, you
    text = re.sub(r'\?\s?\.', '?', text)# ? . -> ?
    text = re.sub(r'\s+', ' ', text)    # strip all redundant whitespace that could have been caused by preprocessing
    
    text = re.sub(r'\s+\?', '?', text)
    text = re.sub(r'\s+,', ',', text)
    text = re.sub(r'\.[\s+\.]+', '. ', text)
    text = re.sub(r'\s+\.', '.', text)
    text2 = text
    try:
        text = normalizer.normalize(text, normalize_entity=False, normalize_url=True)['normalize']
    except Exception:
        print("Exception at:" ,text)
        text = text2
    return text.strip().lower()

In [23]:
dataset_dir = "../dataset/malay-dataset/local-movies-subtitle"
subtitle_text_list = []
file_list = []
for dirpath,subdirs,files in os.walk(dataset_dir):
    for f in files:
        if "ms" in f:
            file_list.append(os.path.join(dirpath,f))

for file in tqdm(file_list): 
    subfile = open(file,'rb')
    lines = subfile.read()
    xml = BeautifulSoup(lines,'lxml-xml')
    subtitle_content = xml.find_all('p')
    subtitle_text = ""
    for sub in subtitle_content:
        subtitle_text = subtitle_text+" "+clean_text(sub.getText())
    #subtitle_text = clean_text(subtitle_text[1:])
    subtitle_text_list.append(subtitle_text)


lines = []
with open('../dataset/malay-dataset/dumping-iium.txt','r') as f:
    lines = f.readlines()
    i = 0
    prev = 0
    len_sentence = 0
    with tqdm(total=len(lines)) as pbar:
        while i < len(lines):
            prev = i
            while len_sentence <= 512 and i < len(lines):
                len_sentence += len(lines[i].split())
                i+=1
                pbar.update(1)
            len_sentence = 0
            stri = clean_text(" ".join(lines[prev:i]))
            subtitle_text_list.append(stri)

import random
random.shuffle(subtitle_text_list)


#subtitle_text_list.extend(lines)
    

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/1139838 [00:00<?, ?it/s]

Exception at: I pray for your health, I pray that Allah will ease your journey, and lastly, I pray for you and yours. p/s, please doakan Nur sekali, moga tabah dan kuat, dan moga digantikan dengan yang lebih baik. Hello, thank you admin kalau my article disiarkan. I am so excited to share my experience buying my first house at age of 24 y/o without any guide from anyone. I am lone ranger in this matter. Nasib baik everything going smooth till the end, huhu. Let me share my own experience and the process how i buy it. First of all, I beli Rumawip Kuchaimas (RM198K), itu je harga rumah yang i mampu beli. Kalau diikutkan i nak je beli rumah yang RM300K sebab ada banyak facilities, tapi apakan daya, ukur baju di badan sendiri. But i am so glad i boleh beli rumah dekat area KL, i ingat dah takde harapan nak beli rumah dekat KL sebab semua harga rumah mampu tengok bukan mampu beli. Alhamdulillah, nasib baik ada Rumawip ni, dapat lah juga merasa beli ruma yg affordable dkt area KL. Sedikit in

In [24]:
len(subtitle_text_list)
comma = period = question = 0
for i in tqdm(subtitle_text_list):
    for j in i:
        if j == ',':
            comma += 1
        elif j == '.':
            period += 1
        elif j == '?':
            question+=1

word_count = 0
for j in tqdm(subtitle_text_list):
    for i in j.split():
        if len(i) == 1 and i not in [',','?','.','!','>','<','(',')','[',']']:
            word_count+=1
        elif len(i) > 1:
            word_count+=1


  0%|          | 0/23680 [00:00<?, ?it/s]

  0%|          | 0/23680 [00:00<?, ?it/s]

In [25]:
from texttable import Texttable
result = Texttable()
result.add_rows([['Word count','period','comma','question'],[word_count, period, comma, question]])

print(result.draw())

+------------+---------+--------+----------+
| Word count | period  | comma  | question |
+============+=========+========+==========+
| 12770947   | 1114980 | 635112 | 64487    |
+------------+---------+--------+----------+


In [26]:
from sklearn.model_selection import train_test_split
for i in range(len(subtitle_text_list)):
    subtitle_text_list[i]+= '\n'
train_malay, test_malay = train_test_split(subtitle_text_list,train_size=0.99)
valid_malay, test_malay = train_test_split(test_malay,train_size=1/3)
print(len(subtitle_text_list),len(train_malay),len(valid_malay),len(test_malay))
with open("../dataset/malay-dataset/train_malay.txt",'w') as f:
    f.writelines(train_malay)

with open("../dataset/malay-dataset/valid_malay.txt",'w') as f:
    f.writelines(valid_malay)

with open("../dataset/malay-dataset/test_malay.txt",'w') as f:
    f.writelines(test_malay)

23680 23443 79 158
